In [181]:
import pandas as pd
import numpy as np
from typing import List

In [182]:
# Тренировочный датасет
data = pd.read_csv('data/train_data.csv')

data.head()

,id,annotation,tags,text
0,doc_001,Светлана из Казани дает частные уроки английск...,"['Начать бизнес', 'Самозанятые', 'Свое дело', ...",## Кто такой самозанятый?\n\nПо закону самозан...
1,doc_002,"Елене назначили социальное пособие на ребенка,...","['Защитить права', 'Банки', 'Банковская карта'...",Первым делом нужно попросить банк проверить ма...
2,doc_003,Самый надежный способ не оказаться в долгах — ...,"['Кредиты', 'Долги', 'Просрочки', 'Ипотека', '...",## Не переоценивайте свои финансовые возможнос...
3,doc_004,"Друзья Александра то и дело хвастаются, что по...","['Инвестиции', 'Ценные бумаги', 'Фондовая бирж...",Просто прийти на биржу и купить ценные бумаги ...
4,doc_005,Вы взяли в микрофинансовой организации заем на...,"['Займы', 'Долги', 'Риски', 'Защитить права']","## МФО больше нет в госреестре. Значит, она за..."


In [183]:
df = data.copy()

## Этап 1: препроцессинг до чанкования

In [184]:
def first_preprocess(df):
    # annotation
    df['annotation'] = df['annotation'].apply(lambda x: '' if x is np.nan else x)

    # tags
    df['tags'] = [row[1:-1].replace("'", "") for row in df['tags']]

    # text
    df['text'] = df['text'].str.replace(r'Обновлено \d{2}\.\d{2}\.\d{4} в \d{2}:\d{2}', '', regex=True)

    return df

In [185]:
df = first_preprocess(df)

df.head()

,id,annotation,tags,text
0,doc_001,Светлана из Казани дает частные уроки английск...,"Начать бизнес, Самозанятые, Свое дело, Налоги",## Кто такой самозанятый?\n\nПо закону самозан...
1,doc_002,"Елене назначили социальное пособие на ребенка,...","Защитить права, Банки, Банковская карта, Риски...",Первым делом нужно попросить банк проверить ма...
2,doc_003,Самый надежный способ не оказаться в долгах — ...,"Кредиты, Долги, Просрочки, Ипотека, Кредитная ...",## Не переоценивайте свои финансовые возможнос...
3,doc_004,"Друзья Александра то и дело хвастаются, что по...","Инвестиции, Ценные бумаги, Фондовая биржа, Акц...",Просто прийти на биржу и купить ценные бумаги ...
4,doc_005,Вы взяли в микрофинансовой организации заем на...,"Займы, Долги, Риски, Защитить права","## МФО больше нет в госреестре. Значит, она за..."


## Этап 2: Обработка text

Есть ответы которые содержат подпункты, начинающиеся с ### - в первой версии оставим это как часть ответа на вопрос ##

In [186]:
df['text'] = df['text'].str.replace('### ', ' ')

df.head()

,id,annotation,tags,text
0,doc_001,Светлана из Казани дает частные уроки английск...,"Начать бизнес, Самозанятые, Свое дело, Налоги",## Кто такой самозанятый?\n\nПо закону самозан...
1,doc_002,"Елене назначили социальное пособие на ребенка,...","Защитить права, Банки, Банковская карта, Риски...",Первым делом нужно попросить банк проверить ма...
2,doc_003,Самый надежный способ не оказаться в долгах — ...,"Кредиты, Долги, Просрочки, Ипотека, Кредитная ...",## Не переоценивайте свои финансовые возможнос...
3,doc_004,"Друзья Александра то и дело хвастаются, что по...","Инвестиции, Ценные бумаги, Фондовая биржа, Акц...",Просто прийти на биржу и купить ценные бумаги ...
4,doc_005,Вы взяли в микрофинансовой организации заем на...,"Займы, Долги, Риски, Защитить права","## МФО больше нет в госреестре. Значит, она за..."


In [188]:
print(df.iloc[0, 3])

## Кто такой самозанятый?

По закону самозанятый — это человек, который платит специальный **налог на профессиональный доход** (НПД). При этом не нужно дополнительно отчислять подоходный налог или налог на прибыль.

Получить статус самозанятого могут россияне и проживающие в РФ граждане Армении, Казахстана, Киргизии, Беларуси и Украины.

Оформить самозанятость вправе даже подростки с 14 лет, если они получили согласие родителей.

## Сколько составляет налог на профессиональный доход?

Есть два вида ставок для самозанятых. Какая именно будет использоваться в вашем случае, зависит от того, кто покупает ваши товары или услуги:

* 4% — если деньги пришли от физического лица;
* 6% — если оплата поступила от юридического лица или индивидуального предпринимателя.

Эти ставки не будут меняться до конца 2028 года.

Ученики Светланы — это в основном взрослые люди, которые хотят подтянуть разговорный английский перед отпуском или командировкой. За урок она берет 1000 рублей. Если Светлана зарегис

1) Разобьем каждый текст на блоки вопрос + ответ

text -> [текст, [вопрос, ответ], [вопрос, ответ]]

In [189]:
def split_text(row:str):
    '''Функция сплитует по вопросу'''
    chunks = row.split('##')
    chunks = [chunk for chunk in chunks if chunk != '']
    return chunks

In [190]:
df['text'] = df['text'].apply(lambda x: split_text(x))

df.head()

,id,annotation,tags,text
0,doc_001,Светлана из Казани дает частные уроки английск...,"Начать бизнес, Самозанятые, Свое дело, Налоги",[ Кто такой самозанятый?\n\nПо закону самозаня...
1,doc_002,"Елене назначили социальное пособие на ребенка,...","Защитить права, Банки, Банковская карта, Риски...",[Первым делом нужно попросить банк проверить м...
2,doc_003,Самый надежный способ не оказаться в долгах — ...,"Кредиты, Долги, Просрочки, Ипотека, Кредитная ...",[ Не переоценивайте свои финансовые возможност...
3,doc_004,"Друзья Александра то и дело хвастаются, что по...","Инвестиции, Ценные бумаги, Фондовая биржа, Акц...",[Просто прийти на биржу и купить ценные бумаги...
4,doc_005,Вы взяли в микрофинансовой организации заем на...,"Займы, Долги, Риски, Защитить права","[ МФО больше нет в госреестре. Значит, она зак..."


2) Найдем вопрос в блоке

In [241]:
def second_preprocess(text: str) -> str:
    '''Удаляет лишние проблемы и переносы строк'''
    text = text.replace('\n', ' ')
    return text.strip()

In [233]:
def parse_question(block:str) -> tuple:
    '''Находит вопрос, извлекает его, удаляет из исходного текста
    
    Args:
        row - один блок до обработки
    Returns:
        tuple, где на 0 позиции вопрос (или '' если вопроса не было), на позиции 1 ответ
    
    '''
    candidats = block.split('\n\n')

    if '?' in candidats[0]:
        question = candidats[0]
        answer = block.replace(question, '')
    else:
        answer = block
        question = ''

    question = second_preprocess(question)
    answer = second_preprocess(answer)
    
    return question, answer

In [235]:
def parse_text(row:list):
    '''Получает на вход документ.
    Обрабатывает каждый блок документа его при помощи parse_question.

    Args:
        row - один документ
    Returns:
        Список с обработанными блоками, где каждый блок это tuple с вопросом и ответом. Вопрос может быть пустым.
    '''
    return [parse_question(bloc) for bloc in row]

In [225]:
# Пример обработки для text, который начинается с вопроса
example = df['text'][0][0]

question, answer = parse_question(example)
print(example, question, answer, sep=f"\n{'='*100}\n")

 Кто такой самозанятый?

По закону самозанятый — это человек, который платит специальный **налог на профессиональный доход** (НПД). При этом не нужно дополнительно отчислять подоходный налог или налог на прибыль.

Получить статус самозанятого могут россияне и проживающие в РФ граждане Армении, Казахстана, Киргизии, Беларуси и Украины.

Оформить самозанятость вправе даже подростки с 14 лет, если они получили согласие родителей.


Кто такой самозанятый?
По закону самозанятый — это человек, который платит специальный **налог на профессиональный доход** (НПД). При этом не нужно дополнительно отчислять подоходный налог или налог на прибыль.  Получить статус самозанятого могут россияне и проживающие в РФ граждане Армении, Казахстана, Киргизии, Беларуси и Украины.  Оформить самозанятость вправе даже подростки с 14 лет, если они получили согласие родителей.


In [228]:
# Пример обработки для text, который не начинается с вопроса
example = df['text'][3][0]

question, answer = parse_question(example)
print(example, question, answer, sep=f"\n{'='*100}\n")

Просто прийти на биржу и купить ценные бумаги невозможно. Александру обязательно понадобится профессиональный посредник — брокер или доверительный управляющий, у которых есть лицензия Банка России.

Брокер совершает сделки по указаниям клиента. В таком случае именно Александр должен будет следить за ситуацией на бирже и принимать решения, когда и какие ценные бумаги покупать и в какой момент их продавать. О других участниках рынка ценных бумаг, без которых инвестору не обойтись, и особенностях их работы читайте в статье «Как устроена фондовая биржа».

Доверительный управляющий самостоятельно распоряжается деньгами или имуществом клиента, чтобы извлечь как можно больше прибыли для их владельца. При таком варианте Александру достаточно определиться, какую сумму он планирует вложить и на какой срок, сколько хотел бы заработать и к каким потерям готов. А затем просто периодически контролировать состояние своего инвестиционного портфеля. Сделки за него будет проводить управляющий.

Еще одно

In [247]:
# Применяем функцию
df['text'] = df['text'].apply(lambda x: parse_text(x))

In [249]:
# Пример 1 - блок с вопросом
df['text'][0][0]

('Кто такой самозанятый?',
 'По закону самозанятый — это человек, который платит специальный **налог на профессиональный доход** (НПД). При этом не нужно дополнительно отчислять подоходный налог или налог на прибыль.  Получить статус самозанятого могут россияне и проживающие в РФ граждане Армении, Казахстана, Киргизии, Беларуси и Украины.  Оформить самозанятость вправе даже подростки с 14 лет, если они получили согласие родителей.')

In [250]:
# Пример 2 - блок без вопроса
df['text'][3][0]

('',
 'Просто прийти на биржу и купить ценные бумаги невозможно. Александру обязательно понадобится профессиональный посредник — брокер или доверительный управляющий, у которых есть лицензия Банка России.  Брокер совершает сделки по указаниям клиента. В таком случае именно Александр должен будет следить за ситуацией на бирже и принимать решения, когда и какие ценные бумаги покупать и в какой момент их продавать. О других участниках рынка ценных бумаг, без которых инвестору не обойтись, и особенностях их работы читайте в статье «Как устроена фондовая биржа».  Доверительный управляющий самостоятельно распоряжается деньгами или имуществом клиента, чтобы извлечь как можно больше прибыли для их владельца. При таком варианте Александру достаточно определиться, какую сумму он планирует вложить и на какой срок, сколько хотел бы заработать и к каким потерям готов. А затем просто периодически контролировать состояние своего инвестиционного портфеля. Сделки за него будет проводить управляющий.  Е

## 3. Пример чанкования

In [279]:
def chunk_text(
        question: str,
        answer: str, 
        chunk_size: int = 166, 
        overlap: float = 0.2
        ) -> List[str]:
    """
    Разбивает текст на чанки фиксированного размера (в словах) с перекрытием.

    Args:
        question: вопрос, который будет добавлен к каждому чанку
        answer: ответ, который будем разбивать на чанки
        chunk_size: количество слов в чанке
        overlap: количество слов перекрытия между чанками

    Returns:
        Список чанков текста
    """
    overlap = int(overlap*chunk_size)
    words = answer.split()
    chunks = []

    start = 0
    while start < len(words):
        # Закладываем длинну чанка как количество слов в чанке без вопроса
        end = start + chunk_size - len(question.split())

        chunk = " ".join(words[start:end])
        q_chunk = f'{question} {chunk}'
        chunks.append(q_chunk)

        # следующий чанк начинается раньше конца текущего — overlap
        start += chunk_size - overlap

        # если достигли конца текста
        if start >= len(words):
            break

    return chunks

In [277]:
# Пример документа, блока
example_doc = df['text'][0]
example_bloc = example_doc[0]

example_bloc

('Кто такой самозанятый?',
 'По закону самозанятый — это человек, который платит специальный **налог на профессиональный доход** (НПД). При этом не нужно дополнительно отчислять подоходный налог или налог на прибыль.  Получить статус самозанятого могут россияне и проживающие в РФ граждане Армении, Казахстана, Киргизии, Беларуси и Украины.  Оформить самозанятость вправе даже подростки с 14 лет, если они получили согласие родителей.')

In [283]:
example_chunk = chunk_text(question=example_bloc[0], answer=example_bloc[1], chunk_size=30)

print(*example_chunk, sep='\n\n')

Кто такой самозанятый? По закону самозанятый — это человек, который платит специальный **налог на профессиональный доход** (НПД). При этом не нужно дополнительно отчислять подоходный налог или налог на прибыль. Получить

Кто такой самозанятый? на прибыль. Получить статус самозанятого могут россияне и проживающие в РФ граждане Армении, Казахстана, Киргизии, Беларуси и Украины. Оформить самозанятость вправе даже подростки с 14 лет, если

Кто такой самозанятый? 14 лет, если они получили согласие родителей.
